# Getting Started: Your First Agent

In this notebook, we'll walk through creating your first Claude agent using the SDK. We'll start simple and progressively add complexity.

## Setup

Before we start, let's set up the notebook environment to run async code properly:

In [22]:
# Setup for running async code in Jupyter
import nest_asyncio
nest_asyncio.apply()

# Load environment variables from .env file if it exists
from dotenv import load_dotenv
load_dotenv()

print("✓ Notebook environment configured")

✓ Notebook environment configured


## Language Support

### SDK Available in Two Languages

The Claude Agent SDK is available in both **Python** and **TypeScript** with full feature parity.

```
Python SDK                        TypeScript SDK

pip install                       npm install
claude-agent-sdk                  @anthropic-ai/claude-agent-sdk
```

### This Course Uses Python

All examples, exercises, and hands-on labs will use Python. The concepts translate directly to TypeScript if you prefer.

| Resource | Link |
|----------|------|
| Python SDK Docs | [platform.claude.com/docs/en/agent-sdk/python](https://platform.claude.com/docs/en/agent-sdk/python) |
| TypeScript SDK Docs | [platform.claude.com/docs/en/agent-sdk/typescript](https://platform.claude.com/docs/en/agent-sdk/typescript) |

## Installation

### Prerequisites

- Python 3.8 or higher
- An Anthropic API key

### Install the SDK

If you're using this repo with `uv`, dependencies are already installed. Otherwise, uncomment and run the cell below:

In [23]:
# Uncomment to install
# !pip install claude-agent-sdk

### Set Your API Key

Make sure you have your API key set. You can either:

1. Set it as an environment variable:
```bash
export ANTHROPIC_API_KEY="your-api-key-here"
```

2. Or create a `.env` file in the project root:
```
ANTHROPIC_API_KEY=your-api-key-here
```

Let's verify the API key is available:

In [24]:
import os

# Check if API key is set
api_key = os.environ.get("ANTHROPIC_API_KEY")
if api_key:
    print(f"✓ API key found (length: {len(api_key)} characters)")
else:
    print("✗ API key not found. Please set ANTHROPIC_API_KEY environment variable or create a .env file.")

✓ API key found (length: 108 characters)


## Your First Agent

### Minimal Example

Let's create the simplest possible agent. This agent will:
1. Import the `query` function - the main entry point
2. Call `query()` with a prompt
3. Stream messages back as the agent works
4. Automatically use appropriate tools (Glob, Bash, etc.)

**That's it** - the agent loop, tools, and context management are all handled for you.

Run the cell below to see it in action:

In [25]:
import asyncio
from claude_agent_sdk import query

async def main():
    async for message in query(prompt="What files are in this directory?"):
        print(message)

await main()

SystemMessage(subtype='init', data={'type': 'system', 'subtype': 'init', 'cwd': '/Users/sajal/projects/oreilly/claude_agent_sdk_course/claude_agent_sdk_course_code/notebooks', 'session_id': '826b7398-dfc7-4951-93d6-648526b5d0cd', 'tools': ['Task', 'TaskOutput', 'Bash', 'Glob', 'Grep', 'ExitPlanMode', 'Read', 'Edit', 'Write', 'NotebookEdit', 'WebFetch', 'TodoWrite', 'WebSearch', 'KillShell', 'AskUserQuestion', 'Skill', 'SlashCommand', 'EnterPlanMode'], 'mcp_servers': [], 'model': 'claude-sonnet-4-5-20250929', 'permissionMode': 'default', 'slash_commands': ['compact', 'context', 'cost', 'init', 'pr-comments', 'release-notes', 'review', 'security-review'], 'apiKeySource': 'ANTHROPIC_API_KEY', 'claude_code_version': '2.0.72', 'output_style': 'default', 'agents': ['general-purpose', 'statusline-setup', 'Explore', 'Plan'], 'skills': [], 'plugins': [], 'uuid': 'cd26a2f2-6058-4423-b6d4-e02263f73fbe'})
AssistantMessage(content=[TextBlock(text="I'll check what files are in the current directory 

### Making Output More Readable

The raw message output above shows all the internal details. Let's create a helper function to print messages in a cleaner, more readable format:

In [26]:
import json

def print_message(message):
    """Pretty print agent messages."""
    msg_type = type(message).__name__
    
    if msg_type == "SystemMessage":
        # Skip system messages for cleaner output
        pass
    
    elif msg_type == "AssistantMessage":
        # Print assistant thinking and tool use
        if hasattr(message, 'content'):
            for block in message.content:
                block_type = type(block).__name__
                if block_type == "TextBlock":
                    print(f"🤖 Assistant: {block.text}")
                elif block_type == "ToolUseBlock":
                    print(f"🔧 Tool: {block.name}")
                    if hasattr(block, 'input'):
                        # Show description first if available
                        if 'description' in block.input:
                            print(f"   → {block.input['description']}")
                        # Show other arguments (excluding description)
                        args = {k: v for k, v in block.input.items() if k != 'description'}
                        if args:
                            print(f"   Arguments: {json.dumps(args, indent=6)}")
    
    elif msg_type == "UserMessage":
        # Print tool results
        if hasattr(message, 'content'):
            for block in message.content:
                block_type = type(block).__name__
                if block_type == "ToolResultBlock":
                    if block.is_error:
                        print(f"❌ Tool Error: {block.content}")
                    else:
                        # Show first 500 chars of result
                        content = str(block.content)
                        if len(content) > 500:
                            content = content[:500] + "..."
                        print(f"📤 Tool Result: {content}")
    
    elif msg_type == "ResultMessage":
        # Only show cost and timing metadata (result text is redundant with last AssistantMessage)
        if hasattr(message, 'total_cost_usd') and hasattr(message, 'duration_ms'):
            print(f"\n💰 Cost: ${message.total_cost_usd:.4f} | ⏱️ Time: {message.duration_ms/1000:.1f}s")

# Test it with a simple query
print("Testing the helper function:\n")
async def test_helper():
    async for message in query(prompt="What files are in this directory?"):
        print_message(message)

await test_helper()

Testing the helper function:

🤖 Assistant: I'll list the files in the current directory for you.
🔧 Tool: Bash
   → List files in current directory
   Arguments: {
      "command": "ls -la"
}
📤 Tool Result: drwxr-xr-x@   - sajal 22 Jan 12:11 .ipynb_checkpoints
.rw-r--r--@ 45k sajal 22 Jan 17:29 00_getting_started.ipynb
.rw-------@ 19k sajal  5 Jan 11:27 01_module1_handson.ipynb
.rw-------@ 25k sajal  5 Jan 11:29 02_module2_handson.ipynb
.rw-------@ 26k sajal  5 Jan 11:31 03_module3_handson.ipynb
🤖 Assistant: This directory contains:

1. **`.ipynb_checkpoints/`** - A hidden directory (created by Jupyter) that stores checkpoint files
2. **`00_getting_started.ipynb`** (45 KB) - A Jupyter notebook, last modified Jan 22
3. **`01_module1_handson.ipynb`** (19 KB) - A Jupyter notebook for Module 1 hands-on exercises
4. **`02_module2_handson.ipynb`** (25 KB) - A Jupyter notebook for Module 2 hands-on exercises
5. **`03_module3_handson.ipynb`** (26 KB) - A Jupyter notebook for Module 3 hands-on e

Much better! The helper function filters out verbose system details and shows:
- 🤖 Assistant thinking
- 🔧 Tool usage with descriptions and arguments
- 📤 Tool results (truncated to 500 chars for readability)
- 💰 Cost and timing metadata

You can use `print_message()` in all the examples below for cleaner output.

## Two Input Modes

### Single Message vs Streaming Input

The SDK supports two distinct ways to interact with agents:

| Mode | Description | Best For |
|------|-------------|----------|
| **Single Message** | One-shot queries | Batch jobs, CI/CD, serverless |
| **Streaming Input** | Persistent interactive session | Interactive apps, multi-turn workflows |

### Quick Comparison

```
Single Message Input          Streaming Input Mode (Recommended)
─────────────────────         ─────────────────────────────────
One-shot tasks           →    Multi-turn conversations
Stateless environments   →    Stateful applications
No images                →    Image attachments supported
Simple automation        →    Interruption & queue handling
```

The example above uses **single message input** - the simplest way to get started.

## Single Message Input

### Continuing Conversations

Chain queries using session management. Try running these cells in sequence:

In [27]:
from claude_agent_sdk import ClaudeAgentOptions

# First query - analyze a file
async def first_query():
    async for msg in query(prompt="List the python notebook files in this directory"):
        print_message(msg)

await first_query()

🤖 Assistant: I'll search for Python notebook files in the current directory.
🔧 Tool: Glob
   Arguments: {
      "pattern": "*.ipynb"
}
📤 Tool Result: /Users/sajal/projects/oreilly/claude_agent_sdk_course/claude_agent_sdk_course_code/notebooks/.ipynb_checkpoints/01_module1_handson-checkpoint.ipynb
/Users/sajal/projects/oreilly/claude_agent_sdk_course/claude_agent_sdk_course_code/notebooks/01_module1_handson.ipynb
/Users/sajal/projects/oreilly/claude_agent_sdk_course/claude_agent_sdk_course_code/notebooks/02_module2_handson.ipynb
/Users/sajal/projects/oreilly/claude_agent_sdk_course/claude_agent_sdk_course_code/notebooks/03_module3_handson.ipyn...
🤖 Assistant: I found **6 Python notebook files** in this directory:

**Main notebooks:**
1. `00_getting_started.ipynb`
2. `01_module1_handson.ipynb`
3. `02_module2_handson.ipynb`
4. `03_module3_handson.ipynb`

**Checkpoint files** (auto-saved versions):
5. `.ipynb_checkpoints/00_getting_started-checkpoint.ipynb`
6. `.ipynb_checkpoints/01_module

In [28]:
# Continue the conversation
async def continue_query():
    async for msg in query(
        prompt="what's in the getting started notebook",
        options=ClaudeAgentOptions(continue_conversation=True)
    ):
        print_message(msg)

await continue_query()

🤖 Assistant: I'll read the getting started notebook to see what's in it.
🔧 Tool: Read
   Arguments: {
      "file_path": "/Users/sajal/projects/oreilly/claude_agent_sdk_course/claude_agent_sdk_course_code/notebooks/00_getting_started.ipynb"
}
📤 Tool Result: [{'text': '<cell id="cell-0"><cell_type>markdown</cell_type># Getting Started: Your First Agent\n\nIn this notebook, we\'ll walk through creating your first Claude agent using the SDK. We\'ll start simple and progressively add complexity.</cell id="cell-0">\n<cell id="cell-1"><cell_type>markdown</cell_type>## Setup\n\nBefore we start, let\'s set up the notebook environment to run async code properly:</cell id="cell-1">\n<cell id="cell-2"># Setup for running async code in Jupyter\nimport nest_asy...
🤖 Assistant: The **Getting Started notebook** (`00_getting_started.ipynb`) is a comprehensive introduction to the Claude Agent SDK. Here's what it contains:

## Main Topics Covered:

### 1. **Setup & Installation**
- Configuring Jupyter f

### Limitations

Single message input does **not** support:
- Image attachments in messages
- Dynamic message queueing
- Real-time interruption
- Hook integration

## Streaming Input Mode

### For Interactive Applications

Streaming input mode creates a **persistent interactive session** where you can:
- Send multiple messages over time
- Maintain conversation context across turns
- Queue messages dynamically
- Handle interruptions

This is the **recommended** approach for building interactive applications.

### How It Works

Use `ClaudeSDKClient` with an async generator that yields messages. The agent maintains state throughout the session and can reference previous messages.

In [29]:
from claude_agent_sdk import ClaudeSDKClient

async def interactive_chat():
    """
    Interactive chat session with the agent.
    The session persists across all your messages, maintaining context.
    """
    
    print("🤖 Interactive Chat Session")
    print("=" * 60)
    print("Chat with the agent about this project!")
    print("The agent will remember context across your messages.")
    print("Type 'exit' to end the session.")
    print("=" * 60)
    print()
    
    # Create a persistent session
    # NOTE: Using bypassPermissions for notebook convenience - auto-approves all tools
    # In production, use permission_mode="default" with canUseTool callback for user approval
    options = ClaudeAgentOptions(
        system_prompt="claude_code",
        allowed_tools=["Read", "Bash", "Grep", "Glob"],
        permission_mode="bypassPermissions",  # ⚠️ Auto-approves all tool uses
        max_turns=20
    )
    
    async with ClaudeSDKClient(options) as client:
        while True:
            # Get user input
            user_input = input("👤 You: ").strip()
            
            if not user_input:
                continue
                
            if user_input.lower() in ['exit', 'quit', 'bye']:
                print("\n👋 Chat session ended!")
                break
            
            print()  # Blank line before agent response
            
            # Create a message generator for this turn
            async def send_message():
                yield {
                    "type": "user",
                    "message": {
                        "role": "user",
                        "content": user_input
                    }
                }
            
            # Send the message and get response
            try:
                await client.query(send_message())
                
                # Display the agent's response
                async for message in client.receive_response():
                    print_message(message)
                    # Stop after we get the final result
                    if type(message).__name__ == "ResultMessage":
                        break
                
                print()  # Blank line before next prompt
                
            except Exception as e:
                print(f"❌ Error: {e}\n")
                break

# Start the chat!
# Try asking questions like:
#   "What files are in this directory?"
#   "Tell me about the first notebook"
#   "How many cells does it have?"
await interactive_chat()

🤖 Interactive Chat Session
Chat with the agent about this project!
The agent will remember context across your messages.
Type 'exit' to end the session.



👤 You:  whats in the getting started notebook? 



🤖 Assistant: I'll help you find and read the getting started notebook. Let me search for it first.
🔧 Tool: Glob
   Arguments: {
      "pattern": "**/getting*started*.ipynb"
}
🔧 Tool: Glob
   Arguments: {
      "pattern": "**/*getting*started*.ipynb"
}
🔧 Tool: Glob
   Arguments: {
      "pattern": "*.ipynb"
}
📤 Tool Result: /Users/sajal/projects/oreilly/claude_agent_sdk_course/claude_agent_sdk_course_code/notebooks/.ipynb_checkpoints/00_getting_started-checkpoint.ipynb
/Users/sajal/projects/oreilly/claude_agent_sdk_course/claude_agent_sdk_course_code/notebooks/00_getting_started.ipynb
📤 Tool Result: No files found
📤 Tool Result: /Users/sajal/projects/oreilly/claude_agent_sdk_course/claude_agent_sdk_course_code/notebooks/.ipynb_checkpoints/01_module1_handson-checkpoint.ipynb
/Users/sajal/projects/oreilly/claude_agent_sdk_course/claude_agent_sdk_course_code/notebooks/01_module1_handson.ipynb
/Users/sajal/projects/oreilly/claude_agent_sdk_course/claude_agent_sdk_course_code/notebooks/02_m

👤 You:  ok based on that notebook can you create a python script that takes an input string as an argument and executes a one off agent with it? save the script inside the notebooks directory



🤖 Assistant: I'll create a Python script that takes an input string and executes a one-off agent with it. Let me first read the notebook to understand the pattern, then create the script.
🔧 Tool: Write
   Arguments: {
      "file_path": "/Users/sajal/projects/oreilly/claude_agent_sdk_course/claude_agent_sdk_course_code/notebooks/run_agent.py",
      "content": "#!/usr/bin/env python3\n\"\"\"\nSimple CLI wrapper for Claude Agent SDK.\nTakes a prompt string and executes a one-off agent query.\n\nUsage:\n    python run_agent.py \"What files are in this directory?\"\n    python run_agent.py \"Review the code in main.py\"\n\"\"\"\n\nimport asyncio\nimport sys\nfrom claude_agent_sdk import query, ClaudeAgentOptions\n\n\ndef print_message(message):\n    \"\"\"Pretty print agent messages.\"\"\"\n    msg_type = type(message).__name__\n\n    if msg_type == \"SystemMessage\":\n        # Skip system messages for cleaner output\n        pass\n\n    elif msg_type == \"AssistantMessage\":\n        #

👤 You:  exit



👋 Chat session ended!


### What Streaming Input Enables

In the example above, notice how:
- The session persisted across **3 separate messages**
- The agent remembered context (could reference "that notebook" from the previous question)
- Messages were queued and processed sequentially
- We used `ClaudeSDKClient` with `message_generator()` to control the flow

| Capability | Description |
|------------|-------------|
| **Image Uploads** | Attach images for visual analysis (not shown but supported) |
| **Queued Messages** | Send multiple messages, process sequentially |
| **Context Persistence** | Agent remembers previous conversation turns |
| **Interruption** | Cancel or redirect mid-conversation |
| **Hooks Support** | Lifecycle hooks for custom behavior |

**Use streaming input mode** when building interactive applications, chatbots, or any multi-turn workflows.

## Configuring Agent Behavior

### Adding Options

You can control what tools the agent can use and how it behaves.

In [30]:
async def configured_agent():
    # NOTE: allowed_tools has a known bug - it doesn't restrict built-in tools!
    # Use disallowed_tools instead to block specific tools (blacklist approach)
    async for message in query(
        prompt="create a hello world python script",
        options=ClaudeAgentOptions(
            # Block write tools
            disallowed_tools=["Bash", "Edit", "Write", "NotebookEdit"],
            permission_mode="bypassPermissions"
        )
    ):
        print_message(message)

await configured_agent()

🤖 Assistant: I'll create a simple "Hello World" Python script for you.

```python
print("Hello, World!")
```

Let me create this file:
🔧 Tool: Bash
   Arguments: {
      "command": "cat > hello_world.py << 'EOF'\nprint(\"Hello, World!\")\nEOF"
}
❌ Tool Error: <tool_use_error>Error: No such tool available: Bash</tool_use_error>
🤖 Assistant: Let me use the appropriate tool to create the file:
🔧 Tool: Write
   Arguments: {
      "file_path": "hello_world.py",
      "content": "print(\"Hello, World!\")\n"
}
❌ Tool Error: <tool_use_error>Error: No such tool available: Write</tool_use_error>
🤖 Assistant: I don't have direct file writing capabilities available in my current tool set. However, I can provide you with the Python script that you can create yourself:

**hello_world.py:**
```python
print("Hello, World!")
```

To create and run this script:

1. **Create the file**: Open a text editor and save the above code as `hello_world.py`

2. **Run it**: Execute it from your terminal with:
   `

### Key Options

| Option | Purpose | Example |
|--------|---------|------|
| `disallowed_tools` | Block specific tools (use this, not allowed_tools!) | `["Bash", "Write", "Edit"]` |
| `allowed_tools` | ⚠️ Has known bug - doesn't restrict built-in tools | Use `disallowed_tools` instead |
| `permission_mode` | Control approval requirements | `"bypassPermissions"` |
| `system_prompt` | Customize agent instructions | `"claude_code"` |
| `resume` | Continue previous session | `"session-id-123"` |

**Note:** The `allowed_tools` parameter has a [known limitation](https://github.com/anthropics/claude-agent-sdk-python/issues/361) where it doesn't restrict built-in tools. Use `disallowed_tools` for a blacklist approach instead.

## System Prompt Configuration

### Important Gotcha

The SDK uses a **minimal system prompt by default** (just tool instructions).

### Get Claude Code Behavior

To get the full Claude Code CLI experience, explicitly configure it:

In [ ]:
async def claude_code_style():
    options = ClaudeAgentOptions(
        system_prompt="claude_code",  
        allowed_tools=["Read", "Glob", "Grep", "Bash"]
    )
    
    async for message in query(
        prompt="Tell me about the structure of this project",
        options=options
    ):
        print_message(message)

# await claude_code_style()

### What `system_prompt="claude_code"` Adds

- Code style and formatting guidelines
- Response tone and verbosity settings
- Security and safety instructions
- Context about the working directory
- Best practices for tool usage